In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import sklearn
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

In [52]:
# encoding specified to avoid errors in reading files

area = pd.read_csv('data/DEC_10_SF1_GCTPH1.US05PR_with_ann.csv', encoding = "ISO-8859-1")
unemp = pd.read_csv('data/Unemployment.csv')
educ = pd.read_excel('data/Education.xls')
pops = pd.read_excel('data/PopulationEstimates.xls')
poverty = pd.read_csv('data/PovertyEstimates.csv')
proptax = pd.read_excel('data/state_prop_taxes.xlsx')
homes = pd.read_csv('data/County_Zhvi_AllHomes.csv', encoding = "ISO-8859-1")
aws_loc = pd.read_excel('data/amazon-warehouses.xlsx')
walmart_loc = pd.read_csv('data/walmart-warehouses.csv')

In [350]:
# Trim area dataset

def split_state(x):
    if len(x.split()) >= 5:
        return x.split()[3]
    else:
        return np.nan

In [351]:
area['state'] = area['Geographic area'].apply(split_state)
columns = ['Geographic area.1','state','area_sq_mi']
area_df = area[columns]
area_df = area_df.rename(columns = {'Geographic area.1':'county'})

In [352]:
print(area_df.shape)
area_df.head()

(3274, 3)


,county,state,area_sq_mi
0,United States,NaN,3796742.23
1,Alabama,NaN,52420.07
2,Autauga County,Alabama,604.39
3,Baldwin County,Alabama,2027.31
4,Barbour County,Alabama,904.52


In [74]:
# Trim unemployment dataset
columns = ['Area_name','State','Unemployment_rate_2010','Unemployment_rate_2017']
unemp_df = unemp[columns]

unemp_df['unemployment_change'] = (unemp_df.Unemployment_rate_2017 - unemp_df.Unemployment_rate_2010)
unemp_df['Area_name'] = unemp_df.Area_name.apply(lambda x: x.split(',')[0])
unemp_df.drop('Unemployment_rate_2010',axis=1,inplace=True)
unemp_df = unemp_df.rename(columns = {'Unemployment_rate_2017':'unemployment_rate','Area_name': 'county', 'State':'state'})

print(unemp_df.shape)
unemp_df.head()

(3194, 4)


/Users/flatironschooldc2/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/flatironschooldc2/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,county,state,unemployment_rate,unemployment_change
0,Autauga County,AL,3.9,-5.0
1,Baldwin County,AL,4.1,-5.9
2,Barbour County,AL,5.8,-6.5
3,Bibb County,AL,4.4,-7.0
4,Blount County,AL,4.0,-5.8


In [71]:
# Trim education dataset
# Will need to exclude one column/engineer due to perfect multicollinearity
columns = ['Area name', 'State', 'less_HS_2013-17', 'HS_2013-17','college_associates_2013-17', 'bachelors_2013-17']
educ_df = educ[columns]
educ_df = educ_df.rename(columns = {'less_HS_2013-17':'less_high_school', 'HS_2013-17': 'high_school','college_associates_2013-17': 'college_or_associates', 'bachelors_2013-17': 'bachelors'})

print(educ_df.shape)
educ_df.head()

(3283, 6)


,Area name,State,less_high_school,high_school,college_or_associates,bachelors
0,United States,US,12.7,27.3,29.1,30.9
1,Alabama,AL,14.7,30.9,29.9,24.5
2,Autauga County,AL,12.3,33.6,29.1,25.0
3,Baldwin County,AL,9.8,27.8,31.7,30.7
4,Barbour County,AL,26.9,35.5,25.5,12.0


In [69]:
# Trim population dataset
columns = ['Area_Name', 'State', 'POP_ESTIMATE_2010', 'POP_ESTIMATE_2017', 'R_INTERNATIONAL_MIG_2017', 'R_NET_MIG_2017']
pop_df = pops[columns]

pop_df['pop_growth'] = (pop_df.POP_ESTIMATE_2017 - pop_df.POP_ESTIMATE_2010)*100/pop_df.POP_ESTIMATE_2010
pop_df.drop('POP_ESTIMATE_2010', axis=1, inplace=True)
pop_df = pop_df.rename(columns = {'POP_ESTIMATE_2017':'population', 'R_INTERNATIONAL_MIG_2017': 'int_migration_rate', 'R_NET_MIG_2017': 'net_migration_rate'})

print(pop_df.shape)
pop_df.head()

(3194, 6)


/Users/flatironschooldc2/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/flatironschooldc2/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Area_Name,State,population,int_migration_rate,net_migration_rate,pop_growth
0,United States,US,325147121,NaN,NaN,5.114679
1,Alabama,AL,4875120,0.655040,1.079070,1.873848
2,Autauga County,AL,55443,-0.036139,1.029950,1.258356
3,Baldwin County,AL,212619,0.537672,22.534622,16.114816
4,Barbour County,AL,25158,0.470801,-25.030896,-7.947311


In [68]:
# Trim poverty dataset
columns = ['Area_Name','State','PCTPOVALL_2017','MEDHHINC_2017']
poverty_df = poverty[columns]
poverty_df = poverty_df.rename(columns = {'PCTPOVALL_2017':'poverty_rate', 'MEDHHINC_2017': 'median_hh_income'})
print(poverty_df.shape)
poverty_df.head()

(3192, 4)


,Area_Name,State,poverty_rate,median_hh_income
0,Autauga County,AL,13.4,"58,343"
1,Baldwin County,AL,10.1,"56,607"
2,Barbour County,AL,33.4,"32,490"
3,Bibb County,AL,20.2,"45,795"
4,Blount County,AL,12.8,"48,253"


In [82]:
# Trimming property tax dataset
columns = ['State','Abbreviation','Average effective property tax rate']
tax_df = proptax[columns]
tax_df.rename(columns = {'Average effective property tax rate':'property_tax_rate'},inplace=True)
print(tax_df.shape)
tax_df.head()

(51, 3)


,State,Abbreviation,property_tax_rate
0,Alabama,AL,0.372825
1,Alaska,AK,1.016092
2,Arizona,AZ,0.643978
3,Arkansas,AR,0.633284
4,California,CA,0.699057


In [93]:
# Trimming home prices dataset
columns = ['RegionName','State','SizeRank','2016-07','2016-08','2016-09','2016-10','2016-11','2016-12','2017-01','2017-02','2017-03','2017-04','2017-05','2017-06']
homes_df = homes[columns]

homes_df['avg_home_price'] = homes_df.iloc[:,3:].mean(axis=1)
homes_df.drop(['2016-07','2016-08','2016-09','2016-10','2016-11','2016-12','2017-01','2017-02','2017-03','2017-04','2017-05','2017-06'],axis=1,inplace=True)

print(homes_df.shape)
homes_df

(2839, 4)


/Users/flatironschooldc2/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,RegionName,State,SizeRank,avg_home_price
0,Los Angeles County,CA,1,560333.416667
1,Cook County,IL,2,225118.333333
2,Harris County,TX,3,178261.166667
3,Maricopa County,AZ,4,242960.916667
4,San Diego County,CA,5,530174.833333
...,...,...,...,...
2834,Camas County,ID,2835,89772.916667
2835,Daggett County,UT,2836,167053.250000
2836,Hinsdale County,CO,2837,320431.250000
2837,Mineral County,CO,2838,240151.750000


In [193]:
# Trimming the amazon warehouse dataset

# Obtaining county from lat./long
counties = []
for lat, long in zip(aws_loc.aw_lat, aws_loc.aw_long):
    location = geolocator.reverse(f"{lat},{long}", timeout=None)
    try:
        counties.append(location.raw['address']['county'])
    except:
        counties.append(np.nan)

/Users/flatironschooldc2/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: `timeout=None` has been passed to a geocoder call. Using default geocoder timeout. In geopy 2.0 the behavior will be different: None will mean "no timeout" instead of "default geocoder timeout". Pass geopy.geocoders.base.DEFAULT_SENTINEL instead of None to get rid of this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


In [244]:
aws_df = aws_loc[['aw_zip_str','aw_lat','aw_long','aw_coord','aw_city','aw_state','amazon_indicator','county']]
aws_df['county'] = counties
# Imputing 'Baltimore County' for 21224 zip code values
aws_df.loc[[62,63,64],'county'] = 'Baltimore County'
aws_df.drop('aw_city',axis=1,inplace=True)

(184, 8)


,aw_zip_str,aw_lat,aw_long,aw_coord,aw_city,aw_state,amazon_indicator,county
0,85043,33.433324,-112.207797,"(33.43, -112.21)",Phoenix,Arizona,1,Maricopa County
1,85338,33.412196,-112.421938,"(33.41, -112.42)",Goodyear,Arizona,1,Maricopa County
2,85043,33.431192,-112.156569,"(33.43, -112.16)",Phoenix,Arizona,1,Maricopa County
3,85043,33.448519,-112.221179,"(33.45, -112.22)",Phoenix,Arizona,1,Maricopa County
4,85043,33.431180,-112.159493,"(33.43, -112.16)",Phoenix,Arizona,1,Maricopa County


In [298]:
print(aws_df.shape)
aws_df.head()

(184, 7)


,aw_zip_str,aw_lat,aw_long,aw_coord,aw_state,amazon_indicator,county
0,85043,33.433324,-112.207797,"(33.43, -112.21)",Arizona,1,Maricopa County
1,85338,33.412196,-112.421938,"(33.41, -112.42)",Arizona,1,Maricopa County
2,85043,33.431192,-112.156569,"(33.43, -112.16)",Arizona,1,Maricopa County
3,85043,33.448519,-112.221179,"(33.45, -112.22)",Arizona,1,Maricopa County
4,85043,33.431180,-112.159493,"(33.43, -112.16)",Arizona,1,Maricopa County


In [122]:
# Obtain county from lat/long
# from geopy.geocoders import Nominatim
# geolocator = Nominatim()

# def get_counties(lat,long):
#     location = geolocator.reverse(f"{lat},{long}", timeout=None)
#     return location.raw['address']['county']
    

/Users/flatironschooldc2/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


In [263]:
# Trimming the walmart warehouse dataset

# Deleting zip-code for Puerto Rico
walmart_df = walmart_loc.drop(44)
walmart_df.reset_index(drop=True)

walmart_df.zip_code = walmart_df.zip_code.astype(str)
walmart_df.dtypes

zip_code        object
wh_indicator     int64
dtype: object

In [264]:
lats = []
longs = []
for i in walmart_df.zip_code:
    if len(i) == 5:
        loc = geolocator.geocode({"country": 'United States of America',"postalcode": i})
        lats.append(loc.latitude)
        longs.append(loc.longitude)
    else:
        loc = geolocator.geocode({"country": 'United States of America',"postalcode": '0'+i})
        lats.append(loc.latitude)
        longs.append(loc.longitude)
    

In [265]:
walmart_df['wh_lat'] = lats
walmart_df['wh_long'] = longs
walmart_df['wh_coord'] = list(zip(lats,longs))

walmart_df.head()

,zip_code,wh_indicator,wh_lat,wh_long,wh_coord
0,27302,1,36.085510,-79.242629,"(36.08550952880476, -79.24262921827149)"
1,98421,1,47.248082,-122.406345,"(47.24808186295794, -122.40634483768773)"
2,77535,1,30.044420,-94.889941,"(30.044419650000002, -94.88994075)"
3,75115,1,32.611767,-96.857492,"(32.6117670212239, -96.8574916216362)"
4,78130,1,29.695254,-98.111411,"(29.69525418211604, -98.1114112958971)"


In [269]:
location.raw['address']

{'house_number': '8576',
 'road': 'Red Oak Street',
 'hamlet': 'North Cucamonga',
 'city': 'Rancho Cucamonga',
 'county': 'San Bernardino County',
 'state': 'California',
 'postcode': '91730',
 'country': 'United States of America',
 'country_code': 'us'}

In [270]:
# Obtaining county from lat./long
counties = []
states = []
for lat, long in zip(walmart_df.wh_lat, walmart_df.wh_long):
    location = geolocator.reverse(f"{lat},{long}", timeout=None)
    try:
        counties.append(location.raw['address']['county'])
        states.append(location.raw['address']['state'])
    except:
        counties.append(np.nan)
        states.append(np.nan)

/Users/flatironschooldc2/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: `timeout=None` has been passed to a geocoder call. Using default geocoder timeout. In geopy 2.0 the behavior will be different: None will mean "no timeout" instead of "default geocoder timeout". Pass geopy.geocoders.base.DEFAULT_SENTINEL instead of None to get rid of this warning.
  """


In [271]:
walmart_df['county'] = counties
walmart_df['state'] = states
walmart_df = walmart_df[['zip_code','county','state','wh_lat','wh_long','wh_coord','wh_indicator']]
walmart_df.head()

,zip_code,county,state,wh_lat,wh_long,wh_coord,wh_indicator
0,27302,Orange County,North Carolina,36.085510,-79.242629,"(36.08550952880476, -79.24262921827149)",1
1,98421,Pierce County,Washington,47.248082,-122.406345,"(47.24808186295794, -122.40634483768773)",1
2,77535,Liberty County,Texas,30.044420,-94.889941,"(30.044419650000002, -94.88994075)",1
3,75115,Dallas County,Texas,32.611767,-96.857492,"(32.6117670212239, -96.8574916216362)",1
4,78130,Comal County,Texas,29.695254,-98.111411,"(29.69525418211604, -98.1114112958971)",1


In [280]:
# Imputing 'James City County, VA' for 23185
# walmart_df.loc[[36],'county'] = 'James City County'
# walmart_df.loc[[36],'state'] = 'Virginia'

In [300]:
print(walmart_df.shape)
walmart_df.head()

(176, 7)


,zip_code,county,state,wh_lat,wh_long,wh_coord,wh_indicator
0,27302,Orange County,North Carolina,36.085510,-79.242629,"(36.08550952880476, -79.24262921827149)",1
1,98421,Pierce County,Washington,47.248082,-122.406345,"(47.24808186295794, -122.40634483768773)",1
2,77535,Liberty County,Texas,30.044420,-94.889941,"(30.044419650000002, -94.88994075)",1
3,75115,Dallas County,Texas,32.611767,-96.857492,"(32.6117670212239, -96.8574916216362)",1
4,78130,Comal County,Texas,29.695254,-98.111411,"(29.69525418211604, -98.1114112958971)",1


## Merge datasets, and remove state level information

In [317]:
# drop rows with missing values
homes_df.dropna(inplace=True)

(2839, 5)


/Users/flatironschooldc2/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


(2834, 5)

In [309]:
# Use homes_df as base dataset
homes_df['state_name'] = homes_df.State.map(inv_map)

/Users/flatironschooldc2/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [302]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Palau': 'PW',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

inv_map = {v: k for k, v in us_state_abbrev.items()}

In [319]:
homes_df.head()

,RegionName,State,SizeRank,avg_home_price,state_name
0,Los Angeles County,CA,1,560333.416667,California
1,Cook County,IL,2,225118.333333,Illinois
2,Harris County,TX,3,178261.166667,Texas
3,Maricopa County,AZ,4,242960.916667,Arizona
4,San Diego County,CA,5,530174.833333,California


In [354]:
area_df.head()

,county,state,area_sq_mi
0,United States,NaN,3796742.23
1,Alabama,NaN,52420.07
2,Autauga County,Alabama,604.39
3,Baldwin County,Alabama,2027.31
4,Barbour County,Alabama,904.52


In [361]:
# merge with area dataset
df1 = pd.merge(homes_df, area_df, how='inner', left_on=['RegionName','state_name'], right_on=['county','state'])
df1.drop(['RegionName','State','state_name'],axis=1,inplace=True)
df1['state_abbrev'] = df1.state.map(us_state_abbrev)
df1 = df1[['county','state','state_abbrev','SizeRank','avg_home_price','area_sq_mi']]
df1.head()

,county,state,state_abbrev,SizeRank,avg_home_price,area_sq_mi
0,Los Angeles County,California,CA,1,560333.416667,4750.94
1,Cook County,Illinois,IL,2,225118.333333,1634.86
2,Harris County,Texas,TX,3,178261.166667,1777.48
3,Maricopa County,Arizona,AZ,4,242960.916667,9224.39
4,San Diego County,California,CA,5,530174.833333,4525.68


In [368]:
# merge with unemployment dataset
df2 = pd.merge(df1, unemp_df, how='inner', left_on=['county','state_abbrev'], right_on=['county','state'])
df2.drop('state_y',axis=1,inplace=True)
df2.rename(columns = {'state_x':'state', 'SizeRank':'size_rank'},inplace=True)
df2.head()

,county,state,state_abbrev,size_rank,avg_home_price,area_sq_mi,unemployment_rate,unemployment_change
0,Los Angeles County,California,CA,1,560333.416667,4750.94,4.8,-7.7
1,Cook County,Illinois,IL,2,225118.333333,1634.86,5.1,-5.8
2,Harris County,Texas,TX,3,178261.166667,1777.48,5.1,-3.2
3,Maricopa County,Arizona,AZ,4,242960.916667,9224.39,4.2,-5.3
4,San Diego County,California,CA,5,530174.833333,4525.68,4.0,-6.8


In [372]:
# merge with education dataset
df3 = pd.merge(df2, educ_df, how='inner', left_on=['county','state_abbrev'], right_on=['Area name','State'])
df3.drop(['Area name', 'State'],axis=1,inplace=True)
print(df3.shape)
df3.head()

(2364, 12)


,county,state,state_abbrev,size_rank,avg_home_price,area_sq_mi,unemployment_rate,unemployment_change,less_high_school,high_school,college_or_associates,bachelors
0,Los Angeles County,California,CA,1,560333.416667,4750.94,4.8,-7.7,21.8,20.7,26.2,31.2
1,Cook County,Illinois,IL,2,225118.333333,1634.86,5.1,-5.8,13.8,23.5,25.5,37.2
2,Harris County,Texas,TX,3,178261.166667,1777.48,5.1,-3.2,19.5,23.3,26.7,30.5
3,Maricopa County,Arizona,AZ,4,242960.916667,9224.39,4.2,-5.3,12.9,22.8,32.9,31.4
4,San Diego County,California,CA,5,530174.833333,4525.68,4.0,-6.8,13.3,18.6,30.7,37.4


In [376]:
# merge with population dataset
df4 = pd.merge(df3, pop_df, how='inner', left_on=['county','state_abbrev'], right_on=['Area_Name','State'])
df4.drop(['Area_Name', 'State'],axis=1,inplace=True)
df4.head()

,county,state,state_abbrev,size_rank,avg_home_price,area_sq_mi,unemployment_rate,unemployment_change,less_high_school,high_school,college_or_associates,bachelors,population,int_migration_rate,net_migration_rate,pop_growth
0,Los Angeles County,California,CA,1,560333.416667,4750.94,4.8,-7.7,21.8,20.7,26.2,31.2,10118759,3.456049,-5.558295,3.016829
1,Cook County,Illinois,IL,2,225118.333333,1634.86,5.1,-5.8,13.8,23.5,25.5,37.2,5204502,3.551055,-9.067036,0.102632
2,Harris County,Texas,TX,3,178261.166667,1777.48,5.1,-3.2,19.5,23.3,26.7,30.5,4664159,7.640949,-2.082565,13.543191
3,Maricopa County,Arizona,AZ,4,242960.916667,9224.39,4.2,-5.3,12.9,22.8,32.9,31.4,4329580,2.284534,11.263849,13.185982
4,San Diego County,California,CA,5,530174.833333,4525.68,4.0,-6.8,13.3,18.6,30.7,37.4,3325468,3.435935,-1.311985,7.158882


In [390]:
# merge with poverty dataset
df5 = pd.merge(df4, poverty_df, how='inner', left_on=['county','state_abbrev'], right_on=['Area_Name','State'])
df5.drop(['Area_Name', 'State'],axis=1,inplace=True)
# Convert to integer
df5['median_hh_income'] = df5.median_hh_income.apply(lambda x: int(x.replace(',','')))
df5.head()

,county,state,state_abbrev,size_rank,avg_home_price,area_sq_mi,unemployment_rate,unemployment_change,less_high_school,high_school,college_or_associates,bachelors,population,int_migration_rate,net_migration_rate,pop_growth,poverty_rate,median_hh_income
0,Los Angeles County,California,CA,1,560333.416667,4750.94,4.8,-7.7,21.8,20.7,26.2,31.2,10118759,3.456049,-5.558295,3.016829,14.9,64912
1,Cook County,Illinois,IL,2,225118.333333,1634.86,5.1,-5.8,13.8,23.5,25.5,37.2,5204502,3.551055,-9.067036,0.102632,14.6,61401
2,Harris County,Texas,TX,3,178261.166667,1777.48,5.1,-3.2,19.5,23.3,26.7,30.5,4664159,7.640949,-2.082565,13.543191,15.9,58664
3,Maricopa County,Arizona,AZ,4,242960.916667,9224.39,4.2,-5.3,12.9,22.8,32.9,31.4,4329580,2.284534,11.263849,13.185982,13.5,62221
4,San Diego County,California,CA,5,530174.833333,4525.68,4.0,-6.8,13.3,18.6,30.7,37.4,3325468,3.435935,-1.311985,7.158882,11.9,76048


In [394]:
# merge with property tax dataset
df6 = pd.merge(df5, tax_df, how='inner', left_on=['state','state_abbrev'], right_on=['State','Abbreviation'])
df6.drop(['Abbreviation', 'State'],axis=1,inplace=True)
df6.head()

,county,state,state_abbrev,size_rank,avg_home_price,area_sq_mi,unemployment_rate,unemployment_change,less_high_school,high_school,college_or_associates,bachelors,population,int_migration_rate,net_migration_rate,pop_growth,poverty_rate,median_hh_income,property_tax_rate
0,Los Angeles County,California,CA,1,560333.416667,4750.94,4.8,-7.7,21.8,20.7,26.2,31.2,10118759,3.456049,-5.558295,3.016829,14.9,64912,0.699057
1,San Diego County,California,CA,5,530174.833333,4525.68,4.0,-6.8,13.3,18.6,30.7,37.4,3325468,3.435935,-1.311985,7.158882,11.9,76048,0.699057
2,Orange County,California,CA,6,670382.666667,948.07,3.5,-6.2,15.3,17.3,28.3,39.1,3179950,3.266434,-2.430615,5.416470,11.5,86031,0.699057
3,Riverside County,California,CA,11,341060.000000,7303.42,5.2,-8.6,18.9,26.4,33.2,21.5,2417224,0.807951,8.932883,9.791375,12.9,63776,0.699057
4,San Bernardino County,California,CA,12,307173.166667,20104.83,4.9,-8.6,20.8,26.2,33.2,19.8,2153203,0.721653,1.506749,5.502104,16.0,60270,0.699057


In [409]:
df6.head()
# df6.isnull().sum()

,county,state,state_abbrev,size_rank,avg_home_price,area_sq_mi,unemployment_rate,unemployment_change,less_high_school,high_school,college_or_associates,bachelors,population,int_migration_rate,net_migration_rate,pop_growth,poverty_rate,median_hh_income,property_tax_rate
0,Los Angeles County,California,CA,1,560333.416667,4750.94,4.8,-7.7,21.8,20.7,26.2,31.2,10118759,3.456049,-5.558295,3.016829,14.9,64912,0.699057
1,San Diego County,California,CA,5,530174.833333,4525.68,4.0,-6.8,13.3,18.6,30.7,37.4,3325468,3.435935,-1.311985,7.158882,11.9,76048,0.699057
2,Orange County,California,CA,6,670382.666667,948.07,3.5,-6.2,15.3,17.3,28.3,39.1,3179950,3.266434,-2.430615,5.416470,11.5,86031,0.699057
3,Riverside County,California,CA,11,341060.000000,7303.42,5.2,-8.6,18.9,26.4,33.2,21.5,2417224,0.807951,8.932883,9.791375,12.9,63776,0.699057
4,San Bernardino County,California,CA,12,307173.166667,20104.83,4.9,-8.6,20.8,26.2,33.2,19.8,2153203,0.721653,1.506749,5.502104,16.0,60270,0.699057


In [424]:
# merge with amazon warehouse dataset
aws_df2 = aws_df[['aw_state','county','amazon_indicator']].drop_duplicates().reset_index(drop=True)
df7 = pd.merge(df6, aws_df2, how='left', left_on=['state','county'], right_on=['aw_state','county'])
df7.drop('aw_state',axis=1,inplace=True)
df7.head()

,county,state,state_abbrev,size_rank,avg_home_price,area_sq_mi,unemployment_rate,unemployment_change,less_high_school,high_school,college_or_associates,bachelors,population,int_migration_rate,net_migration_rate,pop_growth,poverty_rate,median_hh_income,property_tax_rate,amazon_indicator
0,Los Angeles County,California,CA,1,560333.416667,4750.94,4.8,-7.7,21.8,20.7,26.2,31.2,10118759,3.456049,-5.558295,3.016829,14.9,64912,0.699057,NaN
1,San Diego County,California,CA,5,530174.833333,4525.68,4.0,-6.8,13.3,18.6,30.7,37.4,3325468,3.435935,-1.311985,7.158882,11.9,76048,0.699057,NaN
2,Orange County,California,CA,6,670382.666667,948.07,3.5,-6.2,15.3,17.3,28.3,39.1,3179950,3.266434,-2.430615,5.416470,11.5,86031,0.699057,NaN
3,Riverside County,California,CA,11,341060.000000,7303.42,5.2,-8.6,18.9,26.4,33.2,21.5,2417224,0.807951,8.932883,9.791375,12.9,63776,0.699057,1.0
4,San Bernardino County,California,CA,12,307173.166667,20104.83,4.9,-8.6,20.8,26.2,33.2,19.8,2153203,0.721653,1.506749,5.502104,16.0,60270,0.699057,1.0


In [427]:
walmart_df2 = walmart_df[['county','state','wh_indicator']].drop_duplicates().reset_index(drop=True)
walmart_df2

,county,state,wh_indicator
0,Orange County,North Carolina,1
1,Pierce County,Washington,1
2,Liberty County,Texas,1
3,Dallas County,Texas,1
4,Comal County,Texas,1
5,Lackawanna County,Pennsylvania,1
6,Stark County,Ohio,1
7,Merrimack County,New Hampshire,1
8,Mecklenburg County,North Carolina,1
9,Forrest County,Mississippi,1


In [434]:
# merge with walmart warehouse dataset
df8 = pd.merge(df7, walmart_df2, how='left', left_on=['state','county'], right_on=['state','county'])
df8.rename(columns = {'wh_indicator':'walmart_indicator'},inplace=True)
df8.head()

,county,state,state_abbrev,size_rank,avg_home_price,area_sq_mi,unemployment_rate,unemployment_change,less_high_school,high_school,...,bachelors,population,int_migration_rate,net_migration_rate,pop_growth,poverty_rate,median_hh_income,property_tax_rate,amazon_indicator,walmart_indicator
0,Los Angeles County,California,CA,1,560333.416667,4750.94,4.8,-7.7,21.8,20.7,...,31.2,10118759,3.456049,-5.558295,3.016829,14.9,64912,0.699057,NaN,1.0
1,San Diego County,California,CA,5,530174.833333,4525.68,4.0,-6.8,13.3,18.6,...,37.4,3325468,3.435935,-1.311985,7.158882,11.9,76048,0.699057,NaN,NaN
2,Orange County,California,CA,6,670382.666667,948.07,3.5,-6.2,15.3,17.3,...,39.1,3179950,3.266434,-2.430615,5.416470,11.5,86031,0.699057,NaN,NaN
3,Riverside County,California,CA,11,341060.000000,7303.42,5.2,-8.6,18.9,26.4,...,21.5,2417224,0.807951,8.932883,9.791375,12.9,63776,0.699057,1.0,1.0
4,San Bernardino County,California,CA,12,307173.166667,20104.83,4.9,-8.6,20.8,26.2,...,19.8,2153203,0.721653,1.506749,5.502104,16.0,60270,0.699057,1.0,1.0


- clean indicator variables
- calculate pop. density column and delete population/land area columns
- feature engineer/eliminate 1 category variables (education level)
- get lat./long. of each county

In [435]:
# impute 0 values for 'NaN'
def indicator_clean(x):
    if math.isnan(x) == True:
        return 0
    else:
        return 1
df8['amazon_indicator'] = df8.amazon_indicator.apply(indicator_clean)
df8.head()

,county,state,state_abbrev,size_rank,avg_home_price,area_sq_mi,unemployment_rate,unemployment_change,less_high_school,high_school,...,bachelors,population,int_migration_rate,net_migration_rate,pop_growth,poverty_rate,median_hh_income,property_tax_rate,amazon_indicator,walmart_indicator
0,Los Angeles County,California,CA,1,560333.416667,4750.94,4.8,-7.7,21.8,20.7,...,31.2,10118759,3.456049,-5.558295,3.016829,14.9,64912,0.699057,0,1.0
1,San Diego County,California,CA,5,530174.833333,4525.68,4.0,-6.8,13.3,18.6,...,37.4,3325468,3.435935,-1.311985,7.158882,11.9,76048,0.699057,0,NaN
2,Orange County,California,CA,6,670382.666667,948.07,3.5,-6.2,15.3,17.3,...,39.1,3179950,3.266434,-2.430615,5.416470,11.5,86031,0.699057,0,NaN
3,Riverside County,California,CA,11,341060.000000,7303.42,5.2,-8.6,18.9,26.4,...,21.5,2417224,0.807951,8.932883,9.791375,12.9,63776,0.699057,1,1.0
4,San Bernardino County,California,CA,12,307173.166667,20104.83,4.9,-8.6,20.8,26.2,...,19.8,2153203,0.721653,1.506749,5.502104,16.0,60270,0.699057,1,1.0


In [436]:
df8['walmart_indicator'] = df8.walmart_indicator.apply(indicator_clean)
df8.head()

,county,state,state_abbrev,size_rank,avg_home_price,area_sq_mi,unemployment_rate,unemployment_change,less_high_school,high_school,...,bachelors,population,int_migration_rate,net_migration_rate,pop_growth,poverty_rate,median_hh_income,property_tax_rate,amazon_indicator,walmart_indicator
0,Los Angeles County,California,CA,1,560333.416667,4750.94,4.8,-7.7,21.8,20.7,...,31.2,10118759,3.456049,-5.558295,3.016829,14.9,64912,0.699057,0,1
1,San Diego County,California,CA,5,530174.833333,4525.68,4.0,-6.8,13.3,18.6,...,37.4,3325468,3.435935,-1.311985,7.158882,11.9,76048,0.699057,0,0
2,Orange County,California,CA,6,670382.666667,948.07,3.5,-6.2,15.3,17.3,...,39.1,3179950,3.266434,-2.430615,5.416470,11.5,86031,0.699057,0,0
3,Riverside County,California,CA,11,341060.000000,7303.42,5.2,-8.6,18.9,26.4,...,21.5,2417224,0.807951,8.932883,9.791375,12.9,63776,0.699057,1,1
4,San Bernardino County,California,CA,12,307173.166667,20104.83,4.9,-8.6,20.8,26.2,...,19.8,2153203,0.721653,1.506749,5.502104,16.0,60270,0.699057,1,1


In [437]:
# make an overall warehouse indicator column
def indicator(row):
    if row.amazon_indicator ==1 or row.walmart_indicator ==1:
        return 1
    else:
        return 0

df8['warehouse_indicator'] = df8.apply(indicator, axis=1)
df8.head()

,county,state,state_abbrev,size_rank,avg_home_price,area_sq_mi,unemployment_rate,unemployment_change,less_high_school,high_school,...,population,int_migration_rate,net_migration_rate,pop_growth,poverty_rate,median_hh_income,property_tax_rate,amazon_indicator,walmart_indicator,warehouse_indicator
0,Los Angeles County,California,CA,1,560333.416667,4750.94,4.8,-7.7,21.8,20.7,...,10118759,3.456049,-5.558295,3.016829,14.9,64912,0.699057,0,1,1
1,San Diego County,California,CA,5,530174.833333,4525.68,4.0,-6.8,13.3,18.6,...,3325468,3.435935,-1.311985,7.158882,11.9,76048,0.699057,0,0,0
2,Orange County,California,CA,6,670382.666667,948.07,3.5,-6.2,15.3,17.3,...,3179950,3.266434,-2.430615,5.416470,11.5,86031,0.699057,0,0,0
3,Riverside County,California,CA,11,341060.000000,7303.42,5.2,-8.6,18.9,26.4,...,2417224,0.807951,8.932883,9.791375,12.9,63776,0.699057,1,1,1
4,San Bernardino County,California,CA,12,307173.166667,20104.83,4.9,-8.6,20.8,26.2,...,2153203,0.721653,1.506749,5.502104,16.0,60270,0.699057,1,1,1


In [440]:
# 7% of county dataset have a warehouse
df8.warehouse_indicator.value_counts(normalize=True)

0    0.928481
1    0.071519
Name: warehouse_indicator, dtype: float64

In [445]:
df8.rename(columns = {'warehouse_indicator':'warehouse_target'},inplace=True)

In [455]:
df8.head()

,county,state,state_abbrev,size_rank,avg_home_price,area_sq_mi,unemployment_rate,unemployment_change,less_high_school,high_school,college_or_associates,bachelors,population,int_migration_rate,net_migration_rate,pop_growth,poverty_rate,median_hh_income,property_tax_rate,amazon_indicator,walmart_indicator,warehouse_target,pop_density
0,Los Angeles County,California,CA,1,560333.416667,4750.94,4.8,-7.7,21.8,20.7,26.2,31.2,10118759,3.456049,-5.558295,3.016829,14.9,64912,0.699057,0,1,1,2129.843568
1,San Diego County,California,CA,5,530174.833333,4525.68,4.0,-6.8,13.3,18.6,30.7,37.4,3325468,3.435935,-1.311985,7.158882,11.9,76048,0.699057,0,0,0,734.799632
2,Orange County,California,CA,6,670382.666667,948.07,3.5,-6.2,15.3,17.3,28.3,39.1,3179950,3.266434,-2.430615,5.416470,11.5,86031,0.699057,0,0,0,3354.129969
3,Riverside County,California,CA,11,341060.000000,7303.42,5.2,-8.6,18.9,26.4,33.2,21.5,2417224,0.807951,8.932883,9.791375,12.9,63776,0.699057,1,1,1,330.971517
4,San Bernardino County,California,CA,12,307173.166667,20104.83,4.9,-8.6,20.8,26.2,33.2,19.8,2153203,0.721653,1.506749,5.502104,16.0,60270,0.699057,1,1,1,107.098792


In [450]:
# Create population density column
df8['pop_density'] = df8.population/df8.area_sq_mi

In [454]:
# Add lat./long. for each county
# timed out due to too many requests
lats = []
longs = []
for county, state in zip(df8.county, df8.state):
    loc = geolocator.geocode({"country": 'United States of America',"state": state,"county": county})
    lats.append(loc.latitude)
    longs.append(loc.longitude)

In [ ]:
df6['latitude'] = lats
df6['longitude'] = longs

In [457]:
df8.to_csv('data/retail_warehouses.csv')